
## 05 Hyperparameter Tuning
**Objetivo**: Optimizar los hiperparámetros del modelo seleccionado.
**Contenido**:
- Uso de técnicas como Grid Search y Random Search.
- Evaluación del rendimiento del modelo optimizado.


In [1]:
## Importar librerías
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

## Cargar dataset de entranamiento y evaluación

In [2]:

dir_data_processed = '../data/processed'
ruta_archivo_X = os.path.join(dir_data_processed, 'X_train_resampled.csv')
X_train_resampled = pd.read_csv(ruta_archivo_X)

ruta_archivo_y = os.path.join(dir_data_processed, 'y_train_resampled.csv')
y_train_resampled = pd.read_csv(ruta_archivo_y)

ruta_archivo_X = os.path.join(dir_data_processed, 'X_val.csv')
X_val = pd.read_csv(ruta_archivo_X)

ruta_archivo_y = os.path.join(dir_data_processed, 'y_val.csv')
y_val = pd.read_csv(ruta_archivo_y)


In [3]:
X_train_resampled.shape, y_train_resampled.shape

((14604, 21), (14604, 1))

In [4]:
X_val.shape, y_val.shape

((64900, 21), (64900, 1))

## Definir el modelo base

In [6]:

rf = RandomForestClassifier(random_state=42)
y_train_resampled = y_train_resampled.values.ravel()
y_val = y_val.values.ravel()

##  Definir los parámetros para Grid Search y Random Search

In [7]:
# Definir los parámetros que se van a probar en Grid Search
param_grid = {
    'n_estimators': [100, 200, 300],  # Número de árboles en el bosque
    'max_features': ['sqrt', 'log2'],  # Número máximo de features consideradas para dividir un nodo
    'max_depth': [None, 10, 20, 30],  # Profundidad máxima del árbol
    'min_samples_split': [2, 5, 10],  # Número mínimo de muestras requeridas para dividir un nodo interno
    'min_samples_leaf': [1, 2, 4],  # Número mínimo de muestras requeridas para ser una hoja
    'bootstrap': [True, False]  # Método de muestreo de instancias para construir árboles
}

# Inicializar Grid Search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)


In [8]:
# Definir la distribución de parámetros para Random Search
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Inicializar Random Search
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=100, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)


## Entrenar y ajustar hiperparámetros

### Grid Search

In [9]:
grid_search.fit(X_train_resampled, y_train_resampled)
print("Mejores hiperparámetros encontrados por Grid Search:")
print(grid_search.best_params_)


### Random Search

In [ ]:
random_search.fit(X_train_resampled, y_train_resampled)
print("Mejores hiperparámetros encontrados por Random Search:")
print(random_search.best_params_)


## Evaluar el modelo optimizado

In [ ]:
# Función para evaluar el rendimiento del modelo
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

# Evaluación del modelo con los mejores parámetros encontrados por Grid Search
best_grid_model = grid_search.best_estimator_
print("Rendimiento del modelo con mejores parámetros (Grid Search):")
evaluate_model(best_grid_model, X_val, y_val)

# Evaluación del modelo con los mejores parámetros encontrados por Random Search
best_random_model = random_search.best_estimator_
print("Rendimiento del modelo con mejores parámetros (Random Search):")
evaluate_model(best_random_model, X_val, y_val)